In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import cv2
from pycocotools.coco import COCO
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataGenerator(Sequence):
    def __init__(self, coco, img_dir, input_size=(128, 128), batch_size=16, shuffle=True):
        self.coco = coco
        self.img_dir = img_dir
        self.input_size = input_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.img_ids = list(self.coco.imgs.keys())
        self.indexes = np.arange(len(self.img_ids))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.img_ids) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        img_ids = [self.img_ids[k] for k in indexes]

        X, y = self.__data_generation(img_ids)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, img_ids):
        X = np.empty((self.batch_size, *self.input_size, 1))
        y = np.empty((self.batch_size, *self.input_size, 1))

        for i, img_id in enumerate(img_ids):
            img_info = self.coco.loadImgs(img_id)[0]
            img_path = os.path.join(self.img_dir, img_info['file_name'])

            # Read and resize image
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
              raise ValueError(f"Image not found or could not be loaded: {img_path}")
            image = cv2.resize(image, self.input_size)
            image = image / 255.0


            # Initialize mask with zeros
            mask = np.zeros(self.input_size, dtype=np.uint8)

            # Get annotations and update mask
            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            anns = self.coco.loadAnns(ann_ids)

            for ann in anns:
                ann_mask = self.coco.annToMask(ann)
                # Resize annotation mask to match input size
                ann_mask = cv2.resize(ann_mask, self.input_size, interpolation=cv2.INTER_NEAREST)
                mask = np.maximum(mask, ann_mask)

            # Normalize mask
            mask = mask / 255.0

            # Assign data to batch arrays
            X[i, :, :, 0] = image
            y[i, :, :, 0] = mask

        return X, y


data_gen_args = dict(rescale=1./255)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Load COCO annotations
train_annotations_path = '/content/drive/MyDrive/train/_annotations.coco.json'
train_images_dir = '/content/drive/MyDrive/train'
val_annotations_path = '/content/drive/MyDrive/valid/_annotations.coco.json'
val_images_dir = '/content/drive/MyDrive/valid'

train_coco = COCO(train_annotations_path)
val_coco = COCO(val_annotations_path)

# Create data generators
train_generator = DataGenerator(train_coco, train_images_dir, input_size=(128, 128), batch_size=16)
val_generator = DataGenerator(val_coco, val_images_dir, input_size=(128, 128), batch_size=16)


loading annotations into memory...
Done (t=0.42s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [ ]:
from tensorflow.keras.layers import Conv2D, Activation, Multiply
def channel_attention(input_tensor, reduction_ratio=8):
    channels = input_tensor.shape[-1]
    channel_descriptor = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
    channel_descriptor = tf.keras.layers.Dense(channels // reduction_ratio, activation='relu')(channel_descriptor)
    channel_descriptor = tf.keras.layers.Dense(channels, activation='sigmoid')(channel_descriptor)
    channel_descriptor = tf.keras.layers.Reshape((1, 1, channels))(channel_descriptor)
    attention = Multiply()([input_tensor, channel_descriptor])
    return attention

In [ ]:
from tensorflow.keras.layers import Lambda

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, Add, Concatenate
from tensorflow.keras.models import Model

# Define the hybrid model
def hybrid_model(input_size=(128, 128, 1)):
    inputs = Input(input_size)

    # Contraction path (Encoder)
    s1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    s1 = Conv2D(64, 3, activation='relu', padding='same')(s1)
    p1 = MaxPooling2D(pool_size=(2, 2))(s1)

    s2 = Conv2D(128, 3, activation='relu', padding='same')(p1)
    s2 = Conv2D(128, 3, activation='relu', padding='same')(s2)
    p2 = MaxPooling2D(pool_size=(2, 2))(s2)

    s3 = Conv2D(256, 3, activation='relu', padding='same')(p2)
    s3 = Conv2D(256, 3, activation='relu', padding='same')(s3)
    p3 = MaxPooling2D(pool_size=(2, 2))(s3)

    s4 = Conv2D(512, 3, activation='relu', padding='same')(p3)
    s4 = Conv2D(512, 3, activation='relu', padding='same')(s4)
    p4 = MaxPooling2D(pool_size=(2, 2))(s4)

    s5 = Conv2D(1024, 3, activation='relu', padding='same')(p4)
    s5 = Conv2D(1024, 3, activation='relu', padding='same')(s5)
    p5 = MaxPooling2D(pool_size=(2, 2))(s5)

    s6 = Conv2D(2048, 3, activation='relu', padding='same')(p5)
    s6 = Conv2D(2048, 3, activation='relu', padding='same')(s6)
    p6 = MaxPooling2D(pool_size=(2, 2))(s6)

    # Bottleneck
    b1 = Conv2D(4096, 3, activation='relu', padding='same')(p5)
    b1 = Conv2D(4096, 3, activation='relu', padding='same')(b1)

    # Expansion path (Decoder)

    d1 = Conv2D(2048, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(b1))
    d1_a = channel_attention(d1)
    d1_a = Lambda(lambda x: tf.image.resize(x, (4, 4)))(d1_a)

    d1 = Concatenate(axis=3)([s6, d1_a])
    d1 = Conv2D(2048, 3, activation='relu', padding='same')(d1)
    d1 = Conv2D(2048, 3, activation='relu', padding='same')(d1)

    d2 = Conv2D(1024, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(d1))
    d2_a = channel_attention(d2)
    d2 = Concatenate(axis=3)([s5,d2_a])
    d2 = Conv2D(1024, 3, activation='relu', padding='same')(d2)
    d2 = Conv2D(1024, 3, activation='relu', padding='same')(d2)

    d3 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(d2))
    d3_a = channel_attention(d3)
    d3 = Concatenate(axis=3)([s4, d3_a])
    d3 = Conv2D(512, 3, activation='relu', padding='same')(d3)
    d3 = Conv2D(512, 3, activation='relu', padding='same')(d3)

    d4 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(d3))
    d4_a = channel_attention(d4)
    d4 = Add()([s3, d4_a])
    d4 = Conv2D(256, 3, activation='relu', padding='same')(d4)
    d4 = Conv2D(256, 3, activation='relu', padding='same')(d4)

    d5 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(d4))
    d5_a = channel_attention(d5)
    d5 = Add()([s2, d5_a])
    d5 = Conv2D(128, 3, activation='relu', padding='same')(d5)
    d5 = Conv2D(128, 3, activation='relu', padding='same')(d5)

    d6 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(d5))
    d6_a = channel_attention(d6)
    d6 = Add()([s1, d6_a])
    d6 = Conv2D(64, 3, activation='relu', padding='same')(d6)
    d6 = Conv2D(64, 3, activation='relu', padding='same')(d6)
    d6 = Conv2D(2, 3, activation='relu', padding='same')(d6)
    out = Conv2D(1, 1, activation='sigmoid')(d6)

    model = Model(inputs, out)
    return model


# Create the hybrid model
model = hybrid_model(input_size=(128, 128, 1))
model.compile(optimizer= 'Adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()


# Set parameters
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 64)         640       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 64)         36928     ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 64)           0         ['conv2d_1[0][0]']            
 D)                                                                                           

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=100,
    validation_data=val_generator,
    validation_steps=validation_steps
)

Epoch 1/100
24/24 [==============================] - 27s 1s/step - loss: 0.0029 - accuracy: 0.9061 - val_loss: 0.0029 - val_accuracy: 0.9080
Epoch 2/100
24/24 [==============================] - 27s 1s/step - loss: 0.0029 - accuracy: 0.9054 - val_loss: 0.0026 - val_accuracy: 0.9158
Epoch 3/100
24/24 [==============================] - 27s 1s/step - loss: 0.0029 - accuracy: 0.9052 - val_loss: 0.0028 - val_accuracy: 0.9107
Epoch 4/100
24/24 [==============================] - 27s 1s/step - loss: 0.0029 - accuracy: 0.9056 - val_loss: 0.0027 - val_accuracy: 0.9120
Epoch 5/100
24/24 [==============================] - 27s 1s/step - loss: 0.0029 - accuracy: 0.9055 - val_loss: 0.0027 - val_accuracy: 0.9126
Epoch 6/100
24/24 [==============================] - 27s 1s/step - loss: 0.0029 - accuracy: 0.9055 - val_loss: 0.0026 - val_accuracy: 0.9142
Epoch 7/100
24/24 [==============================] - 27s 1s/step - loss: 0.0029 - accuracy: 0.9056 - val_loss: 0.0025 - val_accuracy: 0.9163
Epoch 8/100
2

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-2a904258075f>", line 1, in <cell line: 1>
    history = model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1807, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py", line 832, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/polymo

TypeError: object of type 'NoneType' has no len()